In [383]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd

In [384]:
import pickle

with open('hwdata.pkl', 'rb') as file:
    data = pickle.load(file)

In [385]:
data

{'Vocabulary': array([['after'],
        ['a'],
        ['good'],
        ...,
        ['halitosi'],
        ['random'],
        ['non-random']], dtype=object),
 'XTrain': array([[1., 1., 0., ..., 0., 1., 0.],
        [1., 1., 1., ..., 0., 1., 0.],
        [1., 1., 1., ..., 0., 1., 0.],
        ...,
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 1., 0.]]),
 'yTrain': array([[2],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [1],
        [2],
        [2],
        [2],
        [2],
        [1],
        [2],
        [1],
        [2],
        [1],
        [1],
        [1],
        [2],
        [1],
        [1],
        [1],
        [2],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]

In [386]:
XTrain = data['XTrain']
yTrain = data['yTrain']
XTest = data['XTest']
yTest = data['yTest']

XTraindf = pd.DataFrame(data['XTrain'])
yTraindf = pd.DataFrame(data['yTrain'])
XTestdf = pd.DataFrame(data['XTest'])
yTestdf = pd.DataFrame(data['yTest'])
vocabulary = data['Vocabulary']
#i am assuming that 1 means economist and 2 means onion. 

In [387]:
def NB_YPrior(yTrain):

    size = yTrain.shape[0] # number of rows in yTrain
    
    count1 = 0
    for i in range(size):   # count number of 1's in yTrain or number of economist articles
        if yTrain[i] == 1:
            count1 += 1

    p = count1/size # probability of yTrain being 1

    return p


In [388]:
p = NB_YPrior(yTrain)
p

0.7777777777777778

In [389]:
d = np.zeros([2,XTrain.shape[1]])

def NB_XGivenY(XTrain, yTrain, a=0.001, b=0.9):
    
    size = yTrain.shape[0] # number of rows in yTrain

    yt1 = yTraindf[yTraindf[0] == 1] # yTrain where yTrain = 1 or economist articles
    yt2 = yTraindf[yTraindf[0] == 2] # yTrain where yTrain = 2 or onion articles

    xt1 = XTraindf[yTraindf[0] == 1] # XTrain where yTrain = 1 or economist articles
    xt2 = XTraindf[yTraindf[0] == 2] # XTrain where yTrain = 2 or onion articles

    for i in range(vocabulary.shape[0]):
        count1 = 0
        count2 = 0

        for j in range(xt1.shape[0]):
            if xt1.iloc[j, i] == 1:
                count1 += 1
        
        for k in range(xt2.shape[0]):
            if xt2.iloc[k, i] == 1:
                count2 += 1
        
        d[0,i] = count1
        d[1,i] = count2

    return d

In [390]:
d = NB_XGivenY(XTrain, yTrain)

In [391]:
D = np.zeros([2,XTrain.shape[1]])
a=0.001
b=0.9
for i in range(vocabulary.shape[0]):
    D[0,i] = (d[0,i] + a)/(350 + a + b)
    D[1,i] = (d[1,i] + a)/(100 + a + b)

In [392]:
D

array([[5.69964178e-01, 9.48988461e-01, 2.99232547e-01, ...,
        2.84980664e-06, 3.04932160e-01, 2.84980664e-06],
       [6.73937820e-01, 9.91080366e-01, 4.36080911e-01, ...,
        9.92061526e-03, 3.17152456e-01, 9.91070455e-06]])

In [393]:
import numpy as np

def NB_Classify(D, p, X):
    """
    Predict the labels of X.

    :param D: the probability P(X|Y)
    :param p: the probability P(Y = 1)
    :param X: numpy array of size [num_samples, feat_dim]
             where num_samples is the number of samples
             and feat_dim is the dimension of features

    :return: y: numpy array of size [num_samples, 1] where
                 num_samples is the number of samples
    """
    num_samples, feat_dim = X.shape
    y = np.zeros((num_samples, 1))

    # Log probabilities to avoid underflow
    log_D = np.log(D)
    log_D_not = np.log(1 - D)
    log_p = np.log(p)
    log_p_not = np.log(1 - p)

    for i in range(num_samples):
        # Log likelihoods for each class
        log_likelihood_class1 = np.sum(X[i] * log_D[0] + (1 - X[i]) * log_D_not[0]) + log_p
        log_likelihood_class2 = np.sum(X[i] * log_D[1] + (1 - X[i]) * log_D_not[1]) + log_p_not

        # Predict class with higher log likelihood
        if log_likelihood_class1 > log_likelihood_class2:
            y[i] = 1
        else:
            y[i] = 2

    return y


In [394]:
def NB_Classify2(D, p, X): #This did not work I gave up. was trying an alternate method

    y = np.zeros(X.shape[0])

    p1 = 0
    p2 = 0

    log_p = np.log(p)
    log_pc = np.log(1-p)

    xt1 = X[yTraindf[0] == 1] # XTrain where yTrain = 1 or economist articles
    xt2 = X[yTraindf[0] == 2] # XTrain where yTrain = 2 or onion articles

    t1 = xt1.index.values
    t2 = xt2.index.values

    D1 = D[0]
    D2 = D[1]


    log_d1 = np.log(D1)
    log_d2 = np.log(D2)
    log_d1c = np.log(1 - D1)
    log_d2c = np.log(1 - D2)

    for k in range(X.shape[0]):

        for i in t1:
            p1 += log_d1[i]
            #p2 += log_d1c[i]
        
        p1 += log_p
        
    

        for j in t2:
            p2 += log_d2[j]
           # p1 += log_d1c[j]

        p2 += log_pc

        if p1 >= p2:
            y[k] = 1
        else:
            y[k] = 2


    return y


        

In [395]:
y = NB_Classify(D, p, XTest)
y.shape

(153, 1)

In [397]:
ydf = pd.DataFrame(y)
ydf2 = pd.DataFrame(yTest)
ydf.value_counts()
ydf2.value_counts()

1    103
2     50
Name: count, dtype: int64

In [398]:
def NB_ClassificationAccuracy(yHat, yTruth):
    matchctr = 0
    acc = 0

    for i in range(yHat.shape[0]):
        if(yHat[i] == yTruth[i]):
            matchctr += 1
    
    acc = matchctr/yHat.shape[0]
    return acc

In [399]:
ac = NB_ClassificationAccuracy(y, yTest)
print("accuracy = ", ac)

accuracy =  0.9738562091503268
